In [48]:
!pip install mteb
!pip install --upgrade google-cloud-translate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.3/183.3 kB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.9/230.9 kB 1.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 3.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 4.0 MB/s eta 0:00:00


In [1]:
from mteb import MTEB

/Users/willhathaway/classes/6.s898/mulitlingual-embeddings/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import datasets
import pandas as pd

In [3]:
%env GOOGLE_APPLICATION_CREDENTIALS=/Users/willhathaway/keys/Google/elevated-creek-406303-ab7912cafb53.json


env: GOOGLE_APPLICATION_CREDENTIALS=/Users/willhathaway/keys/elevated-creek-406303-ab7912cafb53.json


In [4]:
sources = ["mteb/reddit-clustering"]



In [7]:
from google.cloud import translate_v2 as translate

def translate_text(text, lang, batch_size=128):

    translate_client = translate.Client()

    # split the text into chunks of batch_size
    chunks = [text[i:i + batch_size] for i in range(0, len(text), batch_size)]
    # will return a sequence of results for each text.
    result = []
    print(len(chunks))
    for chunk in chunks:
        result += translate_client.translate(chunk, target_language=lang)
    result = list(map(lambda x: x['translatedText'], result))
    print(result)
    return result

goal is to basically go through every single row, and then get them translated. the data structure is 

sentences   labels
1
2
3
...
25

In [9]:
# go through all the "sentences" in the dataset, and apply translate to them, and save the results as a new df
def translate_dataset(dataset, lang):
    data = datasets.load_dataset(dataset, split="test")
    print(data.features)
    df = pd.DataFrame()
    for i in range(len(data)):
        # add the translated sentence to the dataframe using concat
        df = pd.concat([df, pd.DataFrame({"sentences": translate_text(data[i]["sentences"], lang), "labels": data[i]["labels"]})])
    return df

out = translate_dataset(sources[0], "es")

{'sentences': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}


KeyboardInterrupt: 

In [36]:
from google.cloud import translate_v2 as translate
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

def translate_text(text_list, lang, batch_size=128):
    translate_client = translate.Client()

    translated_text_list = []
    for sublist in tqdm(text_list):
        # Process each sublist in batches
        chunks = [sublist[i:i + batch_size] for i in range(0, len(sublist), batch_size)]
        
        translated_sublist = []
        for chunk in chunks:
            translations = translate_client.translate(chunk, target_language=lang)
            for translation in translations:
                translated_sublist.append(translation['translatedText'])
        
        translated_text_list.append(translated_sublist)
    
    return translated_text_list

def translate_dataset(dataset_name, lang):
    data = load_dataset(dataset_name, split="test")
    
    sentences_list = [item['sentences'] for item in data]  # List of lists of sentences
    labels_list = [item['labels'] for item in data]        # List of lists of labels

    # Translate sentences in batches
    translated_sentences_list = translate_text(sentences_list, lang)

    # translated sentences is a list of lists, 

    # create a dataframe with the translated sentences and labels
    df = pd.DataFrame({'sentences': translated_sentences_list, 'labels': labels_list})
    return df

a = translate_dataset(sources[0], "es")

100%|██████████| 25/25 [11:46<00:00, 28.25s/it]


In [10]:
data = datasets.load_dataset(sources[0], split="test")

In [13]:
len(data[0]["sentences"])

7194

In [40]:
# save a
a.to_csv("es.csv")

In [32]:
a

([['Anthony Lister, &#39;el Banksy de Brisbane&#39;, declarado culpable de graffiti',
   'Dick Smith ingresa a la administración voluntaria debido a malas ventas',
   'Restauración de daños por inundaciones: adquisición de servicios adecuados',
   '¿Qué es un verdadero AUSSIE? por Peter Drew',
   '&quot;3 días, 17 líneas y 218 estaciones después, la búsqueda para llegar a todas las estaciones de Melbourne está completa. Si no puedes viajar por el mundo, viaja en metro&quot;',
   'Humo del enorme incendio forestal que asoló Yarloop y más de 21.000 hectáreas de tierras de cultivo. Al momento de esta publicación, el incendio continúa.',
   'Pregunta sobre Centrelink y su aplicación',
   'Twitter de Triple J esta mañana.',
   'Continúa la confianza en el mercado inmobiliario de Australia | Realestatecoulisse.com',
   'El culto al gilipollas',
   'El jefe de ALP de Nueva Gales del Sur, Jamie Clements, dimite en medio de presión por acusaciones de acoso sexual',
   'El programa del Canal Die